---
title: "GeoViews 1.5 Release"
date: "2018-05-14"
description: "Release announcement for GeoViews 1.5"
author: "Philipp Rudiger"
categories: [release, geoviews]
image: "images/gv_1.5_collage.png"
aliases:
  - ../../release_1.5.html
---

In [2]:
#| echo: false
import os
import numpy as np
import pandas as pd
import holoviews as hv
import geoviews as gv
import xarray as xr
import cartopy.crs as ccrs
import geoviews.feature as gf
import geoviews.tile_sources as gvts

import datashader as ds
from holoviews.operation.datashader import rasterize

hv.extension('bokeh', 'matplotlib', logo=False, inline=True)

path = os.path.join(os.path.dirname(hv.__file__), '../examples/assets')
gv_path = os.path.join(os.path.dirname(gv.__file__), '../examples/assets')
ds_path = os.path.join(os.path.dirname(ds.__file__), '../examples/data')
%opts Feature 

<img src="./images/gv_1.5_collage.png" width="80%"></img>

We are very pleased to announce the release of [GeoViews](http://geoviews.org) 1.5!

This release contains a large number of features and improvements. Some highlights include:

**Major feature**:

- The bokeh backend now supports arbitrary geographic projections, no longer just Web Mercator ([#170](https://github.com/ioam/geoviews/pull/170))

**New components**:

- Added [``Graph`` element](http://holoviews.org/reference/elements/bokeh/Graph.html) to plot networks of connected nodes ([#115](https://github.com/ioam/geoviews/pull/115))

- Added [``TriMesh`` element](http://holoviews.org/reference/elements/bokeh/TriMesh.html) and datashading operation to plot small and large irregular triangular meshes ([#115](https://github.com/ioam/geoviews/pull/115))

- Added [``QuadMesh`` element](http://holoviews.org/reference/elements/bokeh/QuadMesh.html) and datashading operation to plot small and large, irregular rectilinear and curvilinear meshes ([#116](https://github.com/ioam/geoviews/pull/116))

- Added [``VectorField`` element](http://holoviews.org/reference/elements/bokeh/VectorField.html) and datashading operation to plot small and large quiver plots and other collections of vectors ([#122](https://github.com/ioam/geoviews/pull/122))

- Added [``HexTiles`` element](http://holoviews.org/reference/elements/bokeh/HexTiles.html) to plot data binned into a hexagonal grid ([#147](https://github.com/ioam/geoviews/pull/147))

- Added [``Labels`` element](http://holoviews.org/reference/elements/bokeh/Labels.html) to plot a large number of text labels at once (as data rather than as annotations) ([#147](https://github.com/ioam/geoviews/pull/147))

**New features**:

- Hover tool now supports displaying geographic coordinates as longitude and latitude ([#158](https://github.com/ioam/geoviews/pull/158)) 
    
- Added a new ``geoviews.tile_sources`` module with a predefined set of tile sources ([#165](https://github.com/ioam/geoviews/pull/165))

- Wrapped the xESMF library as a regridding and interpolation operation for rectilinear and curvilinear grids ([#127](https://github.com/ioam/geoviews/pull/127))

- HoloViews operations including ``datashade`` and ``rasterize`` now retain geographic ``crs`` coordinate system ([#118](https://github.com/ioam/geoviews/pull/118))

**Enhancements**:

- Overhauled documentation and added a gallery ([#121](https://github.com/ioam/geoviews/pull/121))
    
Plus many other bug fixes, enhancements and documentation improvements. For full details, see the [Release Notes](https://github.com/ioam/geoviews/releases/tag/v1.5.0).

<hr>

If you are using [Anaconda](https://www.anaconda.com/downloads), GeoViews can most easily be installed by executing the command ``conda install -c pyviz geoviews`` . Otherwise, you can also use ``pip install geoviews`` as long as you satisfy the [cartopy](http://scitools.org.uk/cartopy/) dependency yourself.

<hr>

## Bokeh support for projections

In the past the Bokeh backend for GeoViews only supported displaying plots in Web Mercator coordinates. In this release this limitation was lifted and plots may now be projected to almost all supported Cartopy projections (to see the full list see the [user guide](http://geoviews.org/user_guide/Projections.html)):

In [16]:
cities = pd.read_csv(gv_path+'/cities.csv', encoding="ISO-8859-1")
points = gv.Points(cities[cities.Year==2050], ['Longitude', 'Latitude'], ['City', 'Population'])
features = gf.ocean * gf.land * gf.coastline

options = dict(width=600, height=350, global_extent=True,
               show_bounds=True, color='black', tools=['hover'], axiswise=True,
               color_index='Population', size_index='Population', size=0.002, cmap='viridis')

(features * points.options(projection=ccrs.Mollweide(), **options) +
 features * points.options(projection=ccrs.PlateCarree(), **options))

:Layout
   .Overlay.I  :Overlay
      .Ocean.I     :Feature   [Longitude,Latitude]
      .Land.I      :Feature   [Longitude,Latitude]
      .Coastline.I :Feature   [Longitude,Latitude]
      .Points.I    :Points   [Longitude,Latitude]   (City,Population)
   .Overlay.II :Overlay
      .Ocean.I     :Feature   [Longitude,Latitude]
      .Land.I      :Feature   [Longitude,Latitude]
      .Coastline.I :Feature   [Longitude,Latitude]
      .Points.I    :Points   [Longitude,Latitude]   (City,Population)

## New elements

The other main enhancements to GeoViews in the 1.5 release come from the addition of a wide array of new elements, some of which were recently added in HoloViews and others which have been newly made aware of geographic coordinate systems and added to Geoviews.

### Graph

The first such addition is the new [``Graph``](http://holoviews.org/reference/elements/bokeh/Graph.html) element which was added to HoloViews 1.9 and has now been made aware of geographic coordinates. The example below (available [in the gallery](http://geoviews.org/gallery/bokeh/great_circle.html)) demonstrates how to use the ``Graph`` element to display airport routes from Hawaii with great-circle paths:

In [34]:
#| echo: false
import pyproj
from bokeh.sampledata.airport_routes import airports, routes

def get_circle_path(start, end, samples=200):
    sx, sy = start
    ex, ey = end
    g = pyproj.Geod(ellps='WGS84')
    (az12, az21, dist) = g.inv(sx, sy, ex, ey)
    lonlats = g.npts(sx, sy, ex, ey, 200)
    return np.array([(sx, sy)]+lonlats+[(ex, ey)])

# Compute great-circle paths for all US flight routes from Honolulu
paths = []
honolulu = (-157.9219970703125, 21.318700790405273)
routes = routes[routes.SourceID==3728]
airports = airports[airports.AirportID.isin(list(routes.DestinationID)+[3728])]
for i, route in routes.iterrows():
    airport = airports[airports.AirportID==route.DestinationID].iloc[0]
    paths.append(get_circle_path(honolulu, (airport.Longitude, airport.Latitude)))
tiles = gv.WMTS('https://maps.wikimedia.org/osm-intl/{Z}/{X}/{Y}@2x.png')

# Define Graph from Nodes and EdgePaths
path = gv.EdgePaths(paths)
points = gv.Nodes(airports, ['Longitude', 'Latitude', 'AirportID'], ['Name', 'City'])
graph = gv.Graph((routes, points, path), ['SourceID', 'DestinationID']).options(
    width=600, height=400, tools=['hover', 'tap'], node_fill_color='black', node_size=8
)

graph_example = (tiles * graph).redim.range(Longitude=(-170, -62))

mime = hv.renderer('bokeh').components(graph_example)
with open('images/airport_graph.js', 'w') as f:
    f.write(mime[0]['application/javascript'])

<div id='b48a8c20-927f-4b19-b3ca-b1cb9b2e7cb0' style='display: table; margin: 0 auto;'>
<div class="bk-root">
<div class="bk-plotdiv" id="aee12a22-6460-44bb-ae8d-f5df8af8f1ed"></div>
</div>
</div>

<script async src="./images/airport_graph.js"></script>

### VectorField

Another element that has been available in HoloViews and now been made aware of geographic coordinates is [``VectorField``](http://holoviews.org/reference/elements/bokeh/VectorField.html), useful for displaying vector quantities on a map. Like most HoloViews and GeoViews elements it can be rendered using both Bokeh (left) and Matplotlib (right):

In [3]:
#| echo: false
from cartopy.examples import arrows

def sample_data(shape=(20, 30)):
    """
    Returns ``(x, y, u, v, crs)`` of some vector data
    computed mathematically. The returned crs will be a rotated
    pole CRS, meaning that the vectors will be unevenly spaced in
    regular PlateCarree space.

    """
    crs = ccrs.RotatedPole(pole_longitude=177.5, pole_latitude=37.5)

    x = np.linspace(311.9, 391.1, shape[1])
    y = np.linspace(-23.6, 24.8, shape[0])

    x2d, y2d = np.meshgrid(x, y)
    u = 10 * (2 * np.cos(2 * np.deg2rad(x2d) + 3 * np.deg2rad(y2d + 30)) ** 2)
    v = 20 * np.cos(6 * np.deg2rad(x2d))

    return x, y, u, v, crs

xs, ys, U, V, crs = sample_data()
mag = np.sqrt(U**2 + V**2)
angle = (np.pi/2.) - np.arctan2(U/mag, V/mag)
vectorfield = gv.VectorField((xs, ys, angle, mag), crs=crs)

opts = dict(size_index='Magnitude', show_grid=True, projection=ccrs.Orthographic(-10, 45), fig_inches=(6, 6))
plot = (gf.ocean * gf.coastline * gf.land * vectorfield.options(backend='matplotlib', **opts)).redim.range(Longitude=(-180, 180), Latitude=(-90, 90))
mpl_div = hv.Div(hv.renderer('matplotlib').instance(dpi=120).html(plot))

vectorfield_example = gvts.Wikipedia * vectorfield.options(width=500, height=400, size_index='Magnitude') + mpl_div.options(width=500, height=300)

mime = hv.renderer('bokeh').components(vectorfield_example)
with open('images/vectorfield.js', 'w') as f:
    f.write(mime[0]['application/javascript'])

<div id='40a7c8cb-46ce-4f4f-80a6-2f925d06ca47' style='display: table; margin: 0 auto;'>
<div class="bk-root">
<div class="bk-plotdiv" id="95a2f650-d795-4cd1-a103-0982314e76fd"></div>
</div>
</div>

<script async src="./images/vectorfield.js"></script>

### TriMesh

Also building on the graph capabilities is the [``TriMesh`` element](http://holoviews.org/reference/elements/bokeh/TriMesh.html), which allows defining arbitrary meshes from a set of nodes and a set of simplices (triangles defined as lists of node indexes). The TriMesh element allows easily visualizing Delaunay triangulations and even very large meshes, thanks to corresponding support added to [datashader](http://datashader.org/user_guide/6_Trimesh.html). Below we can see a small ``TriMesh`` displayed as a wire frame and an interpolated datashaded mesh of the Chesapeake Bay containing 1M triangles:

In [6]:
#| echo: false

from holoviews.operation.datashader import datashade

# Some points defining a triangulation over (roughly) Britain.
xy = np.asarray([
    [-0.101, 0.872], [-0.080, 0.883], [-0.069, 0.888], [-0.054, 0.890],
    [-0.045, 0.897], [-0.057, 0.895], [-0.073, 0.900], [-0.087, 0.898],
    [-0.090, 0.904], [-0.069, 0.907], [-0.069, 0.921], [-0.080, 0.919],
    [-0.073, 0.928], [-0.052, 0.930], [-0.048, 0.942], [-0.062, 0.949],
    [-0.054, 0.958], [-0.069, 0.954], [-0.087, 0.952], [-0.087, 0.959],
    [-0.080, 0.966], [-0.085, 0.973], [-0.087, 0.965], [-0.097, 0.965],
    [-0.097, 0.975], [-0.092, 0.984], [-0.101, 0.980], [-0.108, 0.980],
    [-0.104, 0.987], [-0.102, 0.993], [-0.115, 1.001], [-0.099, 0.996],
    [-0.101, 1.007], [-0.090, 1.010], [-0.087, 1.021], [-0.069, 1.021],
    [-0.052, 1.022], [-0.052, 1.017], [-0.069, 1.010], [-0.064, 1.005],
    [-0.048, 1.005], [-0.031, 1.005], [-0.031, 0.996], [-0.040, 0.987],
    [-0.045, 0.980], [-0.052, 0.975], [-0.040, 0.973], [-0.026, 0.968],
    [-0.020, 0.954], [-0.006, 0.947], [ 0.003, 0.935], [ 0.006, 0.926],
    [ 0.005, 0.921], [ 0.022, 0.923], [ 0.033, 0.912], [ 0.029, 0.905],
    [ 0.017, 0.900], [ 0.012, 0.895], [ 0.027, 0.893], [ 0.019, 0.886],
    [ 0.001, 0.883], [-0.012, 0.884], [-0.029, 0.883], [-0.038, 0.879],
    [-0.057, 0.881], [-0.062, 0.876], [-0.078, 0.876], [-0.087, 0.872],
    [-0.030, 0.907], [-0.007, 0.905], [-0.057, 0.916], [-0.025, 0.933],
    [-0.077, 0.990], [-0.059, 0.993]])
# Make lats + lons
x = xy[:, 0]*180/3.14159
y = xy[:, 1]*180/3.14159

# A selected triangulation of the points.
triangles = np.asarray([
    [67, 66,  1], [65,  2, 66], [ 1, 66,  2], [64,  2, 65], [63,  3, 64],
    [60, 59, 57], [ 2, 64,  3], [ 3, 63,  4], [ 0, 67,  1], [62,  4, 63],
    [57, 59, 56], [59, 58, 56], [61, 60, 69], [57, 69, 60], [ 4, 62, 68],
    [ 6,  5,  9], [61, 68, 62], [69, 68, 61], [ 9,  5, 70], [ 6,  8,  7],
    [ 4, 70,  5], [ 8,  6,  9], [56, 69, 57], [69, 56, 52], [70, 10,  9],
    [54, 53, 55], [56, 55, 53], [68, 70,  4], [52, 56, 53], [11, 10, 12],
    [69, 71, 68], [68, 13, 70], [10, 70, 13], [51, 50, 52], [13, 68, 71],
    [52, 71, 69], [12, 10, 13], [71, 52, 50], [71, 14, 13], [50, 49, 71],
    [49, 48, 71], [14, 16, 15], [14, 71, 48], [17, 19, 18], [17, 20, 19],
    [48, 16, 14], [48, 47, 16], [47, 46, 16], [16, 46, 45], [23, 22, 24],
    [21, 24, 22], [17, 16, 45], [20, 17, 45], [21, 25, 24], [27, 26, 28],
    [20, 72, 21], [25, 21, 72], [45, 72, 20], [25, 28, 26], [44, 73, 45],
    [72, 45, 73], [28, 25, 29], [29, 25, 31], [43, 73, 44], [73, 43, 40],
    [72, 73, 39], [72, 31, 25], [42, 40, 43], [31, 30, 29], [39, 73, 40],
    [42, 41, 40], [72, 33, 31], [32, 31, 33], [39, 38, 72], [33, 72, 38],
    [33, 38, 34], [37, 35, 38], [34, 38, 35], [35, 37, 36]])

trimesh = gv.TriMesh((triangles, (x, y)))

fpath = ds_path+'/Chesapeake_and_Delaware_Bays.3dm'
df = pd.read_table(fpath, delim_whitespace=True, header=None, skiprows=1,
                   names=('row_type', 'cmp1', 'cmp2', 'cmp3', 'val'), index_col=1)

e3t = df[df['row_type'] == 'E3T'][['cmp1', 'cmp2', 'cmp3']].values.astype(int) - 1
nd  = df[df['row_type'] == 'ND' ][['cmp1', 'cmp2', 'cmp3']].values.astype(float)
nd[:, 2] *= -1 # Make depth increasing

verts = pd.DataFrame(nd,  columns=['x', 'y', 'z'])
tris  = pd.DataFrame(e3t, columns=['v0', 'v1', 'v2'])

tiles = gv.WMTS('https://maps.wikimedia.org/osm-intl/{Z}/{X}/{Y}@2x.png')

points = gv.operation.project_points(gv.Points(verts, vdims=['z']))

trimesh_example = gf.coastline.options(scale='50m', line_color='red', width=400, height=500, axiswise=True) * trimesh.options(tools=['hover'], axiswise=True) +\
gvts.Wikipedia.options(axiswise=True) * datashade(hv.TriMesh((tris, points)), aggregator=ds.mean('z'), precompute=True).options(width=500, height=500, axiswise=True)

mime = hv.renderer('bokeh').components(trimesh_example)
with open('images/gv_trimesh.js', 'w') as f:
    f.write(mime[0]['application/javascript'])

<div id='983aedf5-7c1c-4e78-863a-763746f0abcd' style='display: table; margin: 0 auto;'>
<div id="fig_983aedf5-7c1c-4e78-863a-763746f0abcd">  
<div class="bk-root">
<div class="bk-plotdiv" id="02252e01-de93-4591-a0d6-56383bc62944"></div>
</div>
</div>
</div>

<script async src="./images/gv_trimesh.js"></script>

### QuadMesh

GeoViews has long had an ``Image`` element that supports regularly sampled, rectilinear meshes similar to matplotlib's ``imshow``. To plot irregularly sampled rectilinear and curvilinear meshes, GeoViews now also has a ``QuadMesh`` element (akin to matplotlib's ``pcolormesh``). Below is a curvilinear mesh loaded from xarray:

In [10]:
#| echo: false
ds = xr.tutorial.load_dataset('rasm')
# Adjust the longitudes so they are in the range -180 to 180
ds.xc.values[ds.xc.values>180] -= 360 

gvds = gv.Dataset(ds).ndloc[::4, ::4, 0].redim.range(Tair=(-25, 25))
qmesh = gv.QuadMesh(gvds.data, ['xc', 'yc'], 'Tair')

qmesh_example = qmesh.options(width=600, height=400, cmap='viridis', colorbar=True, labelled=[]) * gv.feature.coastline.options(apply_ranges=False)

mime = hv.renderer('bokeh').components(qmesh_example)
with open('images/gv_qmesh.js', 'w') as f:
    f.write(mime[0]['application/javascript'])

<div id='e5795e87-2164-44d0-b7dc-492d088064eb' style='display: table; margin: 0 auto;'>
<div class="bk-root">
<div class="bk-plotdiv" id="95bb1fcc-554e-446c-85d0-594f62811ab4"></div>
</div>
</div>

<script async src="./images/gv_qmesh.js"></script>

### HexTiles

Another often requested feature is a hexagonal bin plot, which can be very helpful in visualizing large collections of points. Thanks to the recent addition of a hex tiling glyph in the [bokeh 0.12.15 release](https://bokeh.github.io/blog/2018/3/29/release-0-12-15/) it was straightforward to add this support in the form of a [``HexTiles`` element]((http://holoviews.org/reference/elements/bokeh/HexTiles.html), which supports both simple bin counts and weighted binning, and fixed or variable hex sizes.

Below we can see a ``HexTiles`` plot of ~7 million points representing the NYC population, where each hexagonal bin is scaled and colored by the bin value:

In [5]:
#| echo: false
import dask.dataframe as dd
import geoviews.tile_sources as gvts

extents = (-8243208.293241908, 4953519.586791799, -8215378.42054359, 4997602.857167657)
(x0, y0, x1, y1) = extents

census = dd.read_parquet('/Users/philippjfr/datashader/examples/data/census.parq/')
df = hv.Dataset(census).select(meterswest=(x0, x1), metersnorth=(y0, y1)).data.compute()

hextile_example = gvts.ESRI.clone(extents=extents).options(alpha=0.5) *\
hv.HexTiles(df, ['meterswest', 'metersnorth'], extents=extents).options(
    size_index='Count', xaxis=None, yaxis=None, gridsize=(40, 60),
    width=400, height=600, tools=['hover'], line_color='black', line_width=1,
    title_format='NYC Population'
)

# rendered as part of hv_release_1.10

<div id='8e5ac05b-001d-4111-adf0-ad7a91acc18b' style='display: table; margin: 0 auto;'>
<div class="bk-root">
<div class="bk-plotdiv" id="f8bd495f-f86e-4b08-b707-b498a03050d6"></div>
</div>
</div>

<script async src="./images/hextile_example.js"></script>

### Labels

The existing ``Text`` element allows adding text to a plot, but only one item at a time, which is not suitable for plotting the large collections of text items that many users have been requesting. The new ``Labels`` element provides vectorized text plotting, which is probably most often used to annotate data points or regions of another plot type. Here we select the 20 most populous cities in 2050, plot them using the ``Points`` element, and use the ``Labels`` element to label each point:

In [22]:
#| echo: false
cities = pd.read_csv(gv_path+'/cities.csv', encoding="ISO-8859-1")
population = gv.Dataset(cities, kdims=['City', 'Country', 'Year']).select(Year=2050).sort('Population').iloc[-20:]
points = gv.Points(population, ['Longitude', 'Latitude'], ['City', 'Population', 'Country'])

labels_example =gvts.CartoMidnight *\
points.options(
    width=600, height=550, size_index=3, color_index=3, cmap='viridis',
    is_global=True, size=0.01, title_format='Top 20 Cities with highest population in 2050',
    tools=['hover']
) *\
gv.Labels(points).options(
    text_font_size='8pt', text_color='white'
)

mime = hv.renderer('bokeh').components(labels_example)
with open('images/labels_example.js', 'w') as f:
    f.write(mime[0]['application/javascript'])

<div id='499770c0-a40c-40c2-a947-6c0400a7bc9b' style='display: table; margin: 0 auto;'>
<div class="bk-root">
<div class="bk-plotdiv" id="8eb1c3dc-ec2f-4588-af0f-92054dd03ab4"></div>
</div>
</div>

<script async src="./images/labels_example.js"></script>

## Features

Apart from the new collection of elements that were added, GeoViews 1.5 also comes with an impressive set of new features and enhancements.

### Inbuilt Tile Sources

Since plotting on top of a map tile source is such a common and useful feature, a new ``tile_sources`` module has been added to GeoViews. The new ``geoviews.tile_sources`` module includes a number of commonly used tile sources from CartoDB, Stamen, ESRI, OpenStreetMap and Wikipedia, a small selection of which is shown below:

In [18]:
import geoviews.tile_sources as gvts

(gvts.CartoLight + gvts.CartoEco + gvts.ESRI + gvts.OSM + gvts.StamenTerrain + gvts.Wikipedia).cols(3)

:Layout
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .WMTS.II  :WMTS   [Longitude,Latitude]
   .WMTS.III :WMTS   [Longitude,Latitude]
   .WMTS.IV  :WMTS   [Longitude,Latitude]
   .WMTS.V   :WMTS   [Longitude,Latitude]
   .WMTS.VI  :WMTS   [Longitude,Latitude]

### Datashader & xESMF regridding

When working with mesh and raster data in a geographic context it is frequently useful to regrid the data. In this release we have improved support for regridding and rasterizing rectilinear and curvilinear grids and trimeshes using the [Datashader](http://datashader.org/) and [xESMF](http://xesmf.readthedocs.io/en/latest/) libraries. For a detailed overview of these capabilities see the [user guide](http://geoviews.org/user_guide/Resampling_Grids.html). As a quick summary:

* [Datashader](http://datashader.org/) provides capabilities to quickly rasterize and regrid data of all kinds (``Image``, ``RGB``, ``HSV``, ``QuadMesh``, ``TriMesh``, ``Path``, ``Points`` and ``Contours``)  but does not support complex interpolation and weighting schemes
* [xESMF](http://xesmf.readthedocs.io/en/latest/) can regrid between general recti- and curvi-linear grids (``Image`` and ``QuadMesh``) with all ESMF regridding algorithms, such as bilinear, conservative and nearest neighbour

Below you can see the curvilinear mesh displayed above regridded and interpolated using xESMF:

In [20]:
#| echo: false
ds = xr.tutorial.load_dataset('rasm')
# Adjust the longitudes so they are in the range -180 to 180
ds.xc.values[ds.xc.values>180] -= 360 

gvds = gv.Dataset(ds).ndloc[:, :, 0].redim.range(Tair=(-25, 25))
quadmeshes = gv.QuadMesh(gvds.data, ['xc', 'yc'], 'Tair')

from geoviews.operation.regrid import weighted_regrid
xesmf_example = weighted_regrid(quadmeshes, streams=[]).options(
    cmap='RdBu_r', width=650, height=400, tools=['hover'], labelled=[], colorbar=True
) *\
gv.feature.coastline.options(apply_ranges=False)

mime = hv.renderer('bokeh').components(xesmf_example)
with open('images/xesmf_example.js', 'w') as f:
    f.write(mime[0]['application/javascript'])

Reuse existing file: bilinear_(-179.877, 179.749)_(16.334, 89.638)_400x400.nc


<div id='7faaeb23-ac2c-41fd-bb3e-47eded100612' style='display: table; margin: 0 auto;'>
<div id="fig_7faaeb23-ac2c-41fd-bb3e-47eded100612">
<div class="bk-root">
<div class="bk-plotdiv" id="e4883890-d172-441f-a981-fdc8811911a5"></div>
</div>
</div>
</div>

<script async src="./images/xesmf_example.js"></script>

### Hover now displays lat/lon coordinates

As you may have noticed when hovering over some of the plots in this blog post, the hover tooltips now automatically format coordinates as latitudes and longitudes rather than the previous (and mostly useless) Web Mercator coordinates.

### Operations now CRS aware

In the past when operations defined in HoloViews were applied to GeoViews elements, the coordinate reference system (CRS) of the data was ignored and a HoloViews element was returned. Thanks to the ability to register pre- and post-processors for operations, operations such as ``datashade``, ``rasterize``, ``contours`` and ``bivariate_kde`` will now retain the coordinate system of the data.

As a simple example we will use the ``bivariate_kde`` operation from HoloViews to generate a density map from a set of points. Here the ``PlateCarree`` crs is retained throughout the operation so that the returned ``Contours`` element is appropriately projected on top of the tile source:

In [17]:
from holoviews.operation.stats import bivariate_kde

population = gv.Points(cities[cities.Year==2050], ['Longitude', 'Latitude'], 'Population')

gvts.StamenTerrainRetina * bivariate_kde(population, bandwidth=0.1).options(
    width=500, height=450, show_legend=False, is_global=True
).relabel('Most populous city density map')

:Overlay
   .WMTS.I                                  :WMTS   [Longitude,Latitude]
   .Contours.Most_populous_city_density_map :Contours   [Longitude,Latitude]   (Density)

### Projection operation improved

The ``gv.project`` operation provides a high-level wrapper for projecting all GeoViews element types and now has better handling for polygons and paths as well as all the new element types added in this release.

## Improved documentation & gallery

This release was also accompanied by an overhaul of the existing documentation, specifically an improved [user guide on projections](http://geoviews.org/user_guide/Projections.html) and a whole new [gallery](http://geoviews.org/gallery) with a wide (and expanding) selection of examples.